In [12]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
TRAIN = "/home/jungmo/Documents/autonomous/caffe/bird_image/train/train.txt"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_200.txt"
BIRD_ROOT = "/home/jungmo/Documents/autonomous/caffe/"

BINARYPROTO_PATH = os.path.join(BIRD_ROOT, "db/mean.binaryproto")
NPY_PATH = os.path.join(BIRD_ROOT, "db/mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

problem = open('egg_classified_as_bird.txt', "w")
def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "color32/googlenet_iter_100000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [13]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_200.txt"
evaluate_model("test_32_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_200.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.36 
Confusion Matrix:
[[2434   17   13   36]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.66 
Confusion Matrix:
[[4883   25   27   65]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.11 
Confusion Matrix:
[[6585   29   38   86]
 [   8  698   46   10]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.98 
Confusion Matrix:
[[8377   29   50   96]
 [  23 1321   89   15]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.24 
Confusion Matrix:
[[10800    34    68   106]
 [   23  1321    89    15]
 [    1     0    34     9]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.09 
Confusion Matrix:
[[10800    34    68   106]
 [   23  1321    89    15]
 [   

In [14]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_150.txt"
evaluate_model("test_32_150.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_150.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.88 
Confusion Matrix:
[[2472    3    3   22]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.96 
Confusion Matrix:
[[4948   11    5   36]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.07 
Confusion Matrix:
[[6664   16   12   46]
 [  28  691   36    7]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.62 
Confusion Matrix:
[[8466   19   15   52]
 [  64 1296   71   17]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.91 
Confusion Matrix:
[[10908    21    17    62]
 [   64  1296    71    17]
 [    5     0    35     4]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 93.81 
Confusion Matrix:
[[10908    21    17    62]
 [   64  1296    71    17]
 [  299     0  1868 

In [15]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_140.txt"
evaluate_model("test_32_140.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_140.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 99.00 
Confusion Matrix:
[[2475    4    5   16]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.90 
Confusion Matrix:
[[4945   13    9   33]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.79 
Confusion Matrix:
[[6661   16   11   50]
 [  33  673   47    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.53 
Confusion Matrix:
[[8459   19   14   60]
 [  56 1294   82   16]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.82 
Confusion Matrix:
[[10907    20    17    64]
 [   56  1294    82    16]
 [    8     0    27     9]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 93.10 
Confusion Matrix:
[[10907    20    17    64]
 [   56  1294    82    16]
 [  278     1  1764 

In [16]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_130.txt"
evaluate_model("test_32_130.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_130.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.88 
Confusion Matrix:
[[2472    2    7   19]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.88 
Confusion Matrix:
[[4944    9   12   35]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.95 
Confusion Matrix:
[[6663   15   15   45]
 [  33  683   37    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.51 
Confusion Matrix:
[[8461   17   20   54]
 [  67 1290   76   15]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.83 
Confusion Matrix:
[[10905    20    23    60]
 [   67  1290    76    15]
 [    2     0    34     8]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 93.41 
Confusion Matrix:
[[10905    20    23    60]
 [   67  1290    76    15]
 [  195     0  1817 

In [17]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_120.txt"
evaluate_model("test_32_120.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_120.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.60 
Confusion Matrix:
[[2465    8    5   22]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.70 
Confusion Matrix:
[[4935   13   13   39]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.77 
Confusion Matrix:
[[6650   15   20   53]
 [  33  683   42    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.25 
Confusion Matrix:
[[8446   18   26   62]
 [  64 1279   88   17]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.61 
Confusion Matrix:
[[10889    20    31    68]
 [   64  1279    88    17]
 [    5     0    33     6]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 93.57 
Confusion Matrix:
[[10889    20    31    68]
 [   64  1279    88    17]
 [  235     0  1868 

In [18]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_110.txt"
evaluate_model("test_32_110.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_110.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.80 
Confusion Matrix:
[[2470    8    6   16]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.00 
Confusion Matrix:
[[4950   12    9   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.19 
Confusion Matrix:
[[6680   13   11   34]
 [  33  684   35   10]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.78 
Confusion Matrix:
[[8485   14   13   40]
 [  71 1293   71   13]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 98.00 
Confusion Matrix:
[[10932    16    15    45]
 [   71  1293    71    13]
 [   14     0    25     5]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 91.27 
Confusion Matrix:
[[10932    16    15    45]
 [   71  1293    71    13]
 [  622     3  1465 

In [19]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_100.txt"
evaluate_model("test_32_100.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_32_100.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 99.12 
Confusion Matrix:
[[2478    8    2   12]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.20 
Confusion Matrix:
[[4960   15    4   21]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.32 
Confusion Matrix:
[[6688   18    5   27]
 [  47  686   24    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 98.01 
Confusion Matrix:
[[8495   19    5   33]
 [  85 1306   48    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 98.14 
Confusion Matrix:
[[10946    21     5    36]
 [   85  1306    48     9]
 [   26     0    16     2]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 87.14 
Confusion Matrix:
[[10946    21     5    36]
 [   85  1306    48     9]
 [ 1446     1   819 